In [104]:
# import pandas as pd
# datastatics = {}
# thred_score = {"en": 25, "eu": 25, "jp": 20}
# for dataset_name in ['eu', 'en', 'jp']:
#     datastatics[dataset_name] = {}
#     df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx')
#     final_df = pd.read_excel(f'results/{dataset_name}-final.xlsx')
#     sub_df = df[~df['job_description'].isin(final_df['job_description'])]
#     sub_df = pd.concat([sub_df, final_df])
#     datastatics[dataset_name]['origin_length'] = len(sub_df)
#     # sub_df.to_excel(f'results/{dataset_name}-llm.xlsx', index=None)
#     sub_df = sub_df[sub_df['score']>=thred_score[dataset_name]]
#     sub_df_group = sub_df.groupby('task').agg(list)[['job_description']]
#     sub_df_group['len'] = sub_df_group['job_description'].apply(len)
#     sub_df_group = sub_df_group.sort_values('len', ascending=False)
#     for thred_len in [0, 5, 10, 15, 20, 25, 40]:
#         all_df = sub_df_group[sub_df_group['len']>=thred_len].explode('job_description')[['job_description']].reset_index()
#         if thred_len == 0:
#             all_df.to_excel(f'results/{dataset_name}/{dataset_name}_all.xlsx', index=None)
#             datastatics[dataset_name]['length'] = len(all_df)
#         datastatics[dataset_name][thred_len] = all_df['task'].nunique()
# pd.DataFrame(datastatics)

,eu,en,jp
origin_length,6344,7752,2736
length,4671,5897,2505
0,1028,1413,838
5,314,415,149
10,116,143,49
15,54,56,13
20,30,26,7
25,14,16,3
40,4,1,1


In [66]:
import pandas as pd
new_df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_all.xlsx')
df_ch = pd.read_excel(f'results/{dataset_name}/{dataset_name}_all_ch.xlsx')
ch2jp = {v:i for i,v in zip(new_df['task'], df_ch['任务'])}

In [67]:
df_group = df_ch.groupby('任务').agg(list)
df_group['len'] = df_group['职位描述'].apply(len)
df_group = df_group.sort_values('len', ascending=False).reset_index()
df_group

,任务,职位描述,len
0,与经理讨论业务战略、实践或政策。,"[与客户定义并协调系统开发的要求, 为了创建用户体验，我们深入倾听客户的业务流程和各种要求。...",47
1,分析数据以为运营决策和活动提供信息。,"[小规模PoC数据分析, 负责从数据分析到系统建设的一切工作。, 由于业绩强劲，现大量招聘！...",30
2,与技术人员协商制定设计和操作计划。,"[与主要 IT 公司架构师和顾问合作检查系统实施, 我们将与您一起解决您公司的IT问题！您可...",26
3,制定业务或市场策略。,"[有助于组织战略规划并提高成员的专业水平, 识别新产品增长的问题, 粉丝社区新功能发布和改进...",24
4,检查财务记录以确保遵守政策和法规。,"[文件检查（无电话支持）, 负责评估, 改进措施工作（最终内部审核管理、ISO维护审核）, ...",22
...,...,...,...
833,雇用农业、渔业和林业工人。,[地面）],1
834,通过采访员工、客户和其他人来收集信息。,[营销内容（横幅、传单、宣传册、白皮书等）的设计策划和制作],1
835,选择完成任务所需的资源。,[您将负责开发呼叫中心的生产力测量工具。],1
836,选择满足患者需求的医疗设备。,[分析、推导动作],1


In [120]:
key_word = '检查'
list_set = df_ch[df_ch['任务'].apply(lambda x: key_word in x)]['任务'].unique()
temp = df_group[df_group['任务'].isin(list_set)][['任务', 'len']].reset_index(drop=True)
temp.to_csv('temp.csv',index=None)
content = open('temp.csv', 'r').read() + "\n\n给定上述表格，我希望能对一些任务进行合并，尤其是将数量较少的task合并到数量较高的task，请返回json文件，关键字是需要被合并的任务，字段内容是合并到的task"
with open('temp.csv', 'w') as w:
    w.write(content)

In [121]:
import json
import os
merge_map_ch = {}
for file in os.listdir(f'results/{dataset_name}/merge_json'):
    if file.endswith('json'):
        merge_map_ch.update(json.load(open(f'results/{dataset_name}/merge_json/{file}', 'r')))
merge_map = {ch2jp[i]:ch2jp[v] for i,v in merge_map_ch.items()}
new_df['task'] = new_df['task'].apply(lambda x: merge_map[x] if x in merge_map else x)

import pandas as pd
datastatics = {}
thred_score = {"en": 25, "eu": 25, "jp": 20}
dataset_name = 'jp'

datastatics[dataset_name] = {}
df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx')
final_df = pd.read_excel(f'results/{dataset_name}-final.xlsx')
sub_df = df[~df['job_description'].isin(final_df['job_description'])]
sub_df = pd.concat([sub_df, final_df])

sub_df = pd.merge(sub_df.drop('task', axis=1), new_df, on=['job_description']) 


datastatics[dataset_name]['origin_length'] = len(sub_df)
# sub_df.to_excel(f'results/{dataset_name}-llm.xlsx', index=None)
sub_df = sub_df[sub_df['score']>=thred_score[dataset_name]]


sub_df_group = sub_df.groupby('task').agg(list)[['job_description']]
sub_df_group['len'] = sub_df_group['job_description'].apply(len)
sub_df_group = sub_df_group.sort_values('len', ascending=False)
for thred_len in [0, 5, 10, 15, 20, 25, 40]:
    all_df = sub_df_group[sub_df_group['len']>=thred_len].explode('job_description')[['job_description']].reset_index()
    if thred_len == 0:
        all_df.to_excel(f'results/{dataset_name}/{dataset_name}_all.xlsx', index=None)
        datastatics[dataset_name]['length'] = len(all_df)
    datastatics[dataset_name][thred_len] = all_df['task'].nunique()
    
pd.DataFrame(datastatics)

,jp
origin_length,2505
length,2505
0,595
5,147
10,64
15,31
20,20
25,15
40,4


In [1]:
import pandas as pd
metrics = {}
for dataset_name in ['en', 'eu', 'jp']:
    metrics[dataset_name] = {}
    df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_all.xlsx')
    df_group = df.groupby('task').agg(list)
    df_group['len'] = df_group['job_description'].apply(len)
    df_group = df_group.sort_values('len', ascending=False)
    df_group = df_group[df_group['len']>-10]
    metrics[dataset_name]['class_num>20'] = len(df_group[df_group['len']>=20])
    metrics[dataset_name]['20>class_num>=10'] = len(df_group[(df_group['len']<20) & (df_group['len']>=10)])
    df = df_group[df_group['len']>=10].explode('job_description').reset_index()
    df.to_excel(f"results/{dataset_name}/{dataset_name}_filter.xlsx", index=None)
    metrics[dataset_name]['instance_num'] = len(df)
pd.DataFrame(metrics)

,en,eu,jp
class_num>20,26,30,20
20>class_num>=10,117,86,44
instance_num,2204,1942,1221


In [1]:
import pandas as pd
for dataset_name in ['en', 'eu', 'jp']:
    df = pd.read_excel(f"results/{dataset_name}/{dataset_name}_filter.xlsx")
    llm_df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx')[['job_title', 'job_description', 'task_top5']]
    llm_df.columns = ['job_title','job_description', 'llm_task']
    llm_df = pd.merge(df, llm_df, on=['job_description'])
    bert_df = pd.read_csv(f'data/{dataset_name}/seed0/input0/task_candidate.csv', sep='\t')
    bert_df['bert_task'] = bert_df.apply(lambda x: [x[f"task{i}"] for i in range(100)], axis=1)
    bert_df = bert_df[['job_title', 'job_description',  'bert_task']]
    final_df = pd.merge(llm_df, bert_df, on=['job_title', 'job_description'])
    final_df = final_df[['job_title', 'job_description', 'task', 'llm_task', 'bert_task']]
    final_df.to_csv(f'results/{dataset_name}/{dataset_name}_filter.csv')
final_df

,job_title,job_description,task,llm_task,bert_task
0,バックエンドエンジニア,飲食系の店舗向け音声AI予約サービスの開発をご担当いただきます,ビジネス戦略または市場戦略を策定します。,"['食べ物や飲み物を提供します。', '調理または提供のために食品を準備します。', '灌漑...","[設計または運用テストの結果を解釈します。, 稼働中の機器を監視して故障を検出します。, 食..."
1,【大阪工場(契約社員)】品質保証課 事務,書類のファイリングなどの書類管理,ビジネス戦略または市場戦略を策定します。,"['記録や関連資料を保管します。', '組織または運用の手順を文書化します。', '従業員の...","[顧客のニーズに合わせてエネルギー製品やサービスをカスタマイズします。, 設計図または仕様を..."
2,【全国】「dodaダイレクト」ITコンサルタント・PM(プロジェクトマネージャー)/シニアコ...,組織の戦略立案やメンバーの専門性向上への貢献,ビジネス戦略または市場戦略を策定します。,"['ビジネス戦略または市場戦略を策定します。', 'ビジネスまたは財務データを分析します。'...","[調査レポートまたはインシデントレポートを準備します。, 情報資料を書く。, 顧客またはサー..."
3,プロダクトマネージャー(PdM)/インターネット/Webサービス・ASP,新プロダクトのグロースに向けた課題特定,ビジネス戦略または市場戦略を策定します。,"['ビジネス戦略または市場戦略を策定します。', 'ビジネスまたは財務データを分析します。'...","[プロジェクト情報を他の人に伝えます。, 顧客またはサービスプロバイダーと契約を交渉します。..."
4,プロダクトマネージャー/インターネット/Webサービス・ASP,ファンコミュニティの新機能リリース、改善に向けたプロジェクトマネジメント業務,ビジネス戦略または市場戦略を策定します。,"['ビジネス戦略または市場戦略を策定します。', '他のユーザーと協力してマーケティング戦略...","[ビジネスまたは財務データを分析します。, ビジネス戦略、実践、またはポリシーについてマネー..."
...,...,...,...,...,...
1216,全然遅くない。SESじゃないから時間が無駄にならない【未経験からITエンジニア(プログラマー...,以下の作業を行いながらスキルアップしていただきます,顧客に一般的な情報やサポートを提供します。,"['顧客に一般的な情報やサポートを提供します。', '同僚と協議して業務活動を調整します。'...","[デジタル文書、データ、または画像をフォーマットします。, コンピュータのハードウェアに関す..."
1217,ウェブデザイナー,シェアハウス物件検索サイト「シェアシェア」運営会社,顧客に一般的な情報やサポートを提供します。,"['顧客に一般的な情報やサポートを提供します。', '販売やその他の取引を処理します。', ...","[地質学的または地理学的データを分析します。, ビデオゲームの機能や詳細を設計します。, 運..."
1218,データ入力やExcelを使用した資料作成業務,〇各営業所からの注文書が届くので、情報のデータ入力 〇納品伝票や請求書の作成 〇メール,顧客に一般的な情報やサポートを提供します。,"['顧客に一般的な情報やサポートを提供します。', '人件費や材料費を見積もります。', '...","[コンピュータのハードウェアに関する推奨事項を他の人に提供します。, データまたはドキュメン..."
1219,薬剤耐性問題をデジタルxバイオ技術で解決するスタートアップ/AIエンジニア/データサイエンティスト,精度検証に至るまで一貫してプロジェクトに関わります,顧客に一般的な情報やサポートを提供します。,"['他の専門家と協議して患者のケアを計画します。', '家族と相談して、クライアントの治療計...","[医療活動に関する数値データを計算します。, 運動プログラム、栄養、または器具の使用について..."
